# 1 Information Extraction

Information comes in many shapes and sizes. One important form is structured data, where there is a regular and predictable
organization of entities and relationships. For example, we might be interested in the relation between companies and locations. Given a particular company, we would like to be able to identify the locations where it does business; conversely, given a location, we would like to discover which companies do business in that location. If our data is in tabular form, such as the example table asgiven below, then answering these queries is straightforward.

![1.png](attachment:1.png)


If this location data was stored in Python as a list of tuples (entity, relation, entity), then the question "Which organizations operate in Atlanta?" could be translated as follows:

In [ ]:
 locs = [('Omnicom', 'IN', 'New York'),
('DDB Needham', 'IN', 'New York'),
('Kaplan Thaler Group', 'IN', 'New York'),
('BBDO South', 'IN', 'Atlanta'),
('Georgia-Pacific', 'IN', 'Atlanta')]
query = [e1 for (e1, rel, e2) in locs if e2=='Atlanta']
print(query)


['BBDO South', 'Georgia-Pacific']


we first convert the unstructured data of natural language sentences into the structured data. Then we reap the benefits of powerful query tools such as SQL. This
method of getting meaning from text is called Information Extraction.

Application of Information Extraction

It has many applications, including business intelligence, resume harvesting, media analysis, sentiment detection,
patent search, and email scanning. A particularly important area of current research involves the attempt to extract structured data out of
electronically-available scientific literature, especially in the domain of biology and medicine.


# 1.1 Information Extraction Architecture

shows the architecture for a simple information extraction system. It begins by processing a document using several of the
procedures discussed in 3 and 5.: first, the raw text of the document is split into sentences using a sentence segmenter, and each sentence is further subdivided into words using a tokenizer. Next, each sentence is tagged with part-of-speech tags, which will prove very helpful in the next step, named entity detection. In this step, we search for mentions of potentially interesting entities in each sentence. Finally, we use relation detection to search for likely relations between different entities in the text.

![2.png](attachment:2.png)

To perform the first three tasks, we can define a simple function that simply connects together NLTK's default sentence segmenter, word tokenizer, and part-of-speech tagger :



In [1]:
import nltk, re, pprint
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences.draw()


In [ ]:
ie_preprocess()

# 2 Chunking
The basic technique we will use for entity detection is chunking, which segments and labels multi-token sequences as shown below:

The smaller boxes show the word-level tokenization and part-of-speech tagging, while the large boxes show higher-level chunking.
Each of these larger boxes is called a chunk. Like tokenization, which omits whitespace, chunking usually selects a subset of the tokens.

![image.png](attachment:image.png)




# 2.1 Noun Phrase Chunking

Example of a Simple Regular Expression Based NP Chunker.

In order to create an NP-chunker, we will first define a chunk grammar, consisting of rules that indicate how
sentences should be chunked. In this case, we will define a simple grammar with a single regular-expression rule . This rule says that an NP chunk should be formed whenever the chunker finds an optional determiner (DT) followed by any number of adjectives (JJ) and then a noun (NN). Using this grammar, we create a chunk parser, and test it on our example sentence . The result is a tree, which we can either print, or display graphically.

![3.png](attachment:3.png)



In [ ]:
import nltk
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
("dog", "NN"), ("barked", "VBD"), ("at", "IN"), ("the", "DT"), ("cat", "NN")]
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
print(result)
result.draw()

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


# 2.2 Chinking
Sometimes it is easier to define what we want to exclude from a chunk. We can define a chink to be a sequence of tokens that is not
included in a chunk. In the following example, barked/VBD at/IN is a chink:

[ the/DT little/JJ yellow/JJ dog/NN ] barked/VBD at/IN [ the/DT cat/NN ]

Chinking is the process of removing a sequence of tokens from a chunk. If the matching sequence of tokens spans an entire chunk, then the whole chunk is removed; if the sequence of tokens appears in the middle of the chunk, these tokens are removed, leaving two
chunks where there was only one before. If the sequence is at the periphery of the chunk, these tokens are removed, and a smaller chunk
remains. These three possibilities are illustrated below:



In [ ]:
grammar = r"""
 NP:
 {<.*>+} # Chunk everything
 }<VBD|IN>+{ # Chink sequences of VBD and IN
 """
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
 ("dog", "NN"), ("barked", "VBD"), ("at", "IN"), ("the", "DT"), ("cat", "NN")]
cp = nltk.RegexpParser(grammar)
print(cp.parse(sentence))


(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


# 3 Developing and Evaluating Chunkers

**The IOB format (short for inside, outside, beginning):**

Commonly referred to as the BIO format, is a common tagging format for tagging tokens in a chunking task in computational linguistics (ex. named-entity recognition).

The I- prefix before a tag indicates that the tag is inside a chunk. An O tag indicates that a token belongs to no chunk. The B- prefix before a tag indicates that the tag is the beginning of a chunk that immediately follows another chunk without O tags between them. It is used only in that case: when a chunk comes after an O tag, the first token of the chunk takes the I- prefix.

An example with IOB format:

Alex I-PER
is O
going O
to O
Los I-LOC
Angeles I-LOC
in O
California I-LOC

Let us understand how to evaluate chunkers. As usual, this requires a suitably
annotated corpus. We begin

1) by looking at the mechanics of converting IOB format into an NLTK tree,
2) at how this is done on a larger scale using a chunked corpus.
3) We will see how to score the accuracy of a chunker relative to a corpus


**We can use the NLTK corpus module to access a larger amount of chunked text. The CoNLL 2000 corpus contains 270k words of Wall
Street Journal text, divided into "train" and "test" portions, annotated with part-of-speech tags and chunk tags in the IOB format.**

We can access the data using nltk.corpus.conll2000. Here is an example that reads the 100th sentence of the "train" portion of the corpus:

![5.png](attachment:5.png)

In [ ]:
from nltk.corpus import conll2000
print(conll2000.chunked_sents('train.txt')[99])


As you can see, the CoNLL 2000 corpus contains three chunk types: NP chunks, which we have already seen; VP chunks and PP chunks. Since we are only interested in the NP chunks right now, we can use the
chunk_types argument to select them:

![6.png](attachment:6.png)

# Simple Evaluation and Baselines

Now that we can access a chunked corpus, we can evaluate chunkers. We start off by establishing a baseline for the trivial chunk parser cp that creates no chunks:

Expected output from the following code:

![7.png](attachment:7.png)

The IOB tag accuracy indicates that more than a third of the words are tagged with O, i.e. not in an NP chunk. However, since our tagger
did not find any chunks, its precision, recall, and f-measure are all zero.




In [ ]:
from nltk.corpus import conll2000
cp = nltk.RegexpParser("")
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.evaluate(test_sents))


Now let's try a naive regular expression chunker that looks for
tags beginning with letters that are characteristic of noun phrase tags (e.g. CD, DT, and JJ).

The expected output from the following code is :

![8.png](attachment:8.png)
    
    

In [ ]:
grammar = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
print(cp.evaluate(test_sents))


# 4 Recursion in Linguistic Structure

# 4.1 Building Nested Structure with Cascaded Chunkers

It is possible to build chunk structures of arbitrary depth, simply by creating a multi-stage chunk grammar containing
recursive rules has patterns for noun phrases, prepositional phrases, verb phrases, and sentences. Following exampe is a four-stage chunk
grammar, and can be used to create structures having a depth of at most four.



In [ ]:
import nltk
grammar = r"""
 NP: {<DT|JJ|NN.*>+} # Chunk sequences of DT, JJ, NN
 PP: {<IN><NP>} # Chunk prepositions followed by NP
 VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
 CLAUSE: {<NP><VP>} # Chunk NP, VP
"""
cp = nltk.RegexpParser(grammar)
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"),
 ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))


(S
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


Unfortunately this result misses the VP headed by saw. It has other shortcomings too. Let's see what happens when we apply this
chunker to a sentence having deeper nesting. Notice that it fails to identify the VP chunk.

In [ ]:
sentence = [("John", "NNP"), ("thinks", "VBZ"), ("Mary", "NN"),
("saw", "VBD"), ("the", "DT"), ("cat", "NN"), ("sit", "VB"),
("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))


(S
  (NP John/NNP)
  thinks/VBZ
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


The solution to these problems is to get the chunker to loop over its patterns: after trying all of them, it repeats the process. We add an
optional second argument loop to specify the number of times the set of patterns should be run:


In [ ]:
cp = nltk.RegexpParser(grammar, loop=2)
print(cp.parse(sentence))

(S
  (NP John/NNP)
  thinks/VBZ
  (CLAUSE
    (NP Mary/NN)
    (VP
      saw/VBD
      (CLAUSE
        (NP the/DT cat/NN)
        (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))))


# 4.2 Trees

A tree is a set of connected labeled nodes, each reachable by a unique path from a distinguished root node. Here's an example of a tree
![9.png](attachment:9.png)

We use a 'family' metaphor to talk about the relationships of nodes in a tree: for example, S is the parent of VP; conversely VP is a child
of S. Also, since NP and VP are both children of S, they are also siblings.

Although we will focus on syntactic trees, trees can be used to encode any homogeneous hierarchical structure that spans a sequence of
linguistic forms (e.g. morphological structure, discourse structure). In the general case, leaves and node values do not have to be strings.
In NLTK, we create a tree by giving a node label and a list of children:



In [ ]:
tree1 = nltk.Tree('NP', ['Alice'])
print(tree1)


(NP Alice)


In [ ]:
tree2 = nltk.Tree('NP', ['the', 'rabbit'])
print(tree2)


(NP the rabbit)


In [ ]:
tree3 = nltk.Tree('VP', ['chased', tree2])
tree4 = nltk.Tree('S', [tree1, tree3])
print(tree4)

(S (NP Alice) (VP chased (NP the rabbit)))


In [ ]:
 print(tree4[1])


(VP chased (NP the rabbit))


In [ ]:
tree4[1].label()



'VP'

In [ ]:
 tree4.leaves()


['Alice', 'chased', 'the', 'rabbit']

In [ ]:
tree4[1][1][1]

'rabbit'

The bracketed representation for complex trees can be difficult to read. In these cases, the draw method can be very useful. It opens a
new window, containing a graphical representation of the tree. The tree display window allows you to zoom in and out, to collapse and
expand subtrees, and to print the graphical representation to a postscript file (for inclusion in a document).

In [ ]:
tree3.draw()

# 4.3 Tree Traversal
It is standard to use a recursive function to traverse a tree.  Here is an example of a Recursive Function to Traverse a Tree.

In [ ]:
import nltk
def traverse(t):

#Try to access the label of the current node.
#If it's not possible (AttributeError is raised), print the node itself:
 try:
     t.label()
 except AttributeError:
     print(t, end=" ")
#If accessing the label is successful, print the opening parenthesis and the label:
 else:
 # Now we know that t.node is defined
     print('(', t.label(), end=" ")

#Iterate through each child of the current node and recursively call
#the traverse function for each child:
 for child in t:
     traverse(child)

#After traversing all children, print the closing parenthesis:
print(')', end=" ")

#Define an NLTK tree t representing the structure of a sentence:

t = nltk.Tree('(S (NP Alice) (VP chased (NP the rabbit)))')

#Call the traverse function on the tree t to print the labels of its nodes in a specific format:
traverse(t)

) 

TypeError: Tree: Expected a node value and child list 

# 5 Named Entity Recognition
At the start of this chapter, we briefly introduced named entities (NEs). Named entities are definite noun phrases that refer to specific
types of individuals, such as organizations, persons, dates, and so on. 5.1 lists some of the more commonly used types of NEs. These
should be self-explanatory, except for "Facility": human-made artifacts in the domains of architecture and civil engineering; and "GPE":
geo-political entities such as city, state/province, and country.

![10.png](attachment:10.png)

The goal of a named entity recognition (NER) system is to identify all textual mentions of the named entities. This can be broken
down into two sub-tasks: identifying the boundaries of the NE, and identifying its type. While named entity recognition is frequently a
prelude to identifying relations in Information Extraction, it can also contribute to other tasks. For example, in Question Answering
(QA), we try to improve the precision of Information Retrieval by recovering not whole pages, but just those parts which contain an
answer to the user's question. Most QA systems take the documents returned by standard Information Retrieval, and then attempt to
isolate the minimal text snippet in the document containing the answer. Now suppose the question was Who was the first President of
the US?, and one of the documents that was retrieved contained the following passage:
(5) The Washington Monument is the most prominent structure in Washington, D.C. and one of the city's early attractions. It was
built in honor of George Washington, who led the country to independence and then became its first President.
Analysis of the question leads us to expect that an answer should be of the form X was the first President of the US, where X is not only
a noun phrase, but also refers to a named entity of type PERSON. This should allow us to ignore the first sentence in the passage. While
it contains two occurrences of Washington, named entity recognition should tell us that neither of them has the correct type.
How do we go about identifying named entities? One option would be to look up each word in an appropriate list of names. For
example, in the case of locations, we could use a gazetteer, or geographical dictionary, such as the Alexandria Gazetteer or the Getty
Gazetteer. However, doing this blindly runs into problems, as shown below:






![11.png](attachment:11.png)

Observe that the gazetteer has good coverage of locations in many countries, and incorrectly finds locations like Sanchez in the
Dominican Republic and On in Vietnam. Of course we could omit such locations from the gazetteer, but then we won't be able to
identify them when they do appear in a document.
It gets even harder in the case of names for people or organizations. Any list of such names will probably have poor coverage. New
organizations come into existence every day, so if we are trying to deal with contemporary newswire or blog entries, it is unlikely that
we will be able to recognize many of the entities using gazetteer lookup.
Another major source of difficulty is caused by the fact that many named entity terms are ambiguous. Thus May and North are likely to
be parts of named entities for DATE and LOCATION, respectively, but could both be part of a PERSON; conversely Christian Dior
looks like a PERSON but is more likely to be of type ORGANIZATION. A term like Yankee will be ordinary modifier in some contexts,
but will be marked as an entity of type ORGANIZATION in the phrase Yankee infielders.

Further challenges are posed by multi-word names like Stanford University, and by names that contain other names such as Cecil H.
Green Library and Escondido Village Conference Service Center. In named entity recognition, therefore, we need to be able to identify
the beginning and end of multi-token sequences.
Named entity recognition is a task that is well-suited to the type of classifier-based approach that we saw for noun phrase chunking. In
particular, we can build a tagger that labels each word in a sentence using the IOB format, where chunks are labeled by their appropriate
type. Here is part of the CONLL 2002 (conll2002) Dutch training data:

    
NLTK provides a classifier that has already been trained to recognize named entities, accessed with the function nltk.ne_chunk(). If we
set the parameter binary=True , then named entities are just tagged as NE; otherwise, the classifier adds category labels such as
PERSON, ORGANIZATION, and GPE.


In [ ]:
sent = nltk.corpus.treebank.tagged_sents()[22]
print(nltk.ne_chunk(sent, binary=True))

(S
  The/DT
  (NE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (NE Brooke/NNP)
  T./NNP
  Mossman/NNP
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (NE University/NNP)
  of/IN
  (NE Vermont/NNP College/NNP)
  of/IN
  (NE Medicine/NNP)
  ./.)


In [ ]:
 print(nltk.ne_chunk(sent))


(S
  The/DT
  (GPE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (PERSON Brooke/NNP T./NNP Mossman/NNP)
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (ORGANIZATION University/NNP)
  of/IN
  (PERSON Vermont/NNP College/NNP)
  of/IN
  (GPE Medicine/NNP)
  ./.)


# 6 Relation Extraction

Once named entities have been identified in a text, we then want to extract the relations that exist between them. As indicated earlier,
we will typically be looking for relations between specified types of named entity. One way of approaching this task is to initially look
for all triples of the form (X, α, Y), where X and Y are named entities of the required types, and α is the string of words that intervenes
between X and Y. We can then use regular expressions to pull out just those instances of α that express the relation that we are looking
for. The following example searches for strings that contain the word in. The special regular expression (?!\b.+ing\b) is a negative
lookahead assertion that allows us to disregard strings such as success in supervising the transition of, where in is followed by a gerund.


In [ ]:
import nltk, re, pprint
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
        corpus='ieer', pattern = IN):
        print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


Searching for the keyword in works reasonably well, though it will also retrieve false positives such as [ORG: House
Transportation Committee] , secured the most money in the [LOC: New
York]; there is unlikely to be simple string-based method of excluding filler strings such as this.
As shown above, the conll2002 Dutch corpus contains not just named entity annotation but also part-of-speech tags. This allows us to
devise patterns that are sensitive to these tags, as shown in the next example. The method clause() prints out the relations in a clausal
form, where the binary relation symbol is specified as the value of parameter relsym

In [ ]:
import nltk, re, pprint
from nltk.corpus import conll2002
vnv = """
(
is/V| # 3rd sing present and
was/V| # past forms of the verb zijn ('be')
werd/V| # and also present
wordt/V # past of worden ('become)
)
.* # followed by anything
van/Prep # followed by van ('of')
"""
VAN = re.compile(vnv, re.VERBOSE)
for doc in conll2002.chunked_sents('ned.train'):
    for rel in nltk.sem.extract_rels('PER', 'ORG', doc,corpus='conll2002', pattern=VAN):
        print(nltk.sem.clause(rel, relsym="VAN"))

VAN("cornet_d'elzius", 'buitenlandse_handel')
VAN('johan_rottiers', 'kardinaal_van_roey_instituut')
VAN('annie_lennox', 'eurythmics')
